In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/239.5 MB 55.3 kB/s eta 0:45:00m

In [ ]:
(train_img, train_labels), (test_img, test_labels) = fashion_mnist.load_data()

target_classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

sample = []
for id in range(10):
    index = np.where(train_labels == id)[0][0]
    sample.append(train_img[index])

fig, axes = plt.subplots(4, 3, figsize=(5, 5))
fig.suptitle("Fashion-MNIST Sample Images", fontsize=14)

for i, ax in enumerate(axes.flat):
    if i < len(sample):
        ax.imshow(sample[i], cmap='gray')
        ax.set_title(target_classes[i])
        ax.axis("off")
    else:
        ax.axis("off")


plt.tight_layout()
plt.show()
